In [1]:
import asyncio
import os
import httpx
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [2]:
model_client = OpenAIChatCompletionClient(
    model="llama3.2",
    base_url="http://localhost:11434/v1",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
    },
    seed=42, 
    temperature=0,
)

In [3]:
weather_api_url = 'http://api.weatherapi.com/v1/current.json'
params = {
    'key': os.environ['WEATHER_API_KEY'],
    'aqi': 'no'
}

In [4]:
def get_weather(location: str) -> str:
    params.update({'q': location})
    response = httpx.get(weather_api_url, params=params)
    return response.json()

In [7]:
async def main() -> None:
    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant. You can call tools to help user.",
        model_client=model_client,
        tools=[get_weather],
        reflect_on_tool_use=False, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    )
    weather_reporter = AssistantAgent(
        name="weather_reporter",
        system_message="You are a Weather Reporter. You read weather condition and summarize it.",
        model_client=model_client,
    )
    
    while True:
        user_input = input("User: ")
        if user_input == "exit":
            break
        response = await assistant.on_messages([TextMessage(content=user_input, source="user")], CancellationToken())

        weather_summary = await weather_reporter.on_messages([TextMessage(content=response.chat_message.content, source="user")], CancellationToken())
        print("Weather Reporter:", weather_summary.chat_message.content)

await main()

User:  Cumming, GA


Weather Reporter: Here's a summary of the current weather conditions in Cumming, Georgia:

**Current Weather Conditions:**

* Temperature: 13.3°C (55.9°F)
* Condition: Clear
* Wind Speed: 4 km/h (NW direction)
* Humidity: 49%
* Precipitation: 0 mm (no rain)
* Pressure: 1016 mb

Overall, it's a clear and calm night in Cumming with a gentle breeze from the northwest.


User:  Charlotte, NC


Weather Reporter: Here's a summary of the current weather conditions in Charlotte, North Carolina:

**Current Weather Conditions:**

* Temperature: 11.1°C (52.0°F)
* Condition: Clear
* Wind Speed: 4 km/h (SSW direction)
* Humidity: 41%
* Precipitation: 0 mm (no rain)
* Pressure: 1016 mb

Overall, it's a clear and calm night in Charlotte with a gentle breeze from the southwest. The temperature is relatively cool compared to Cumming, Georgia, which reported a higher temperature of 13.3°C (55.9°F).


User:  exit
